In [1]:
import os
import subprocess
def check_mig_with_smi():
    try:
        output = subprocess.check_output(["nvidia-smi", "-L"], text=True)
        return "MIG" in output
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

if check_mig_with_smi():
    print("Running on a GPU MIG instance")
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
else:
    print("Not running on a GPU MIG instance")

Not running on a GPU MIG instance


In [6]:
import platform
print(f"Platform: {platform.system()} {platform.release()}")

Platform: Windows 10


In [2]:
import torch

from modelComp.FluidGPT_B import FluidGPT_B

# load yaml file
yaml_path = "../conf/model/std-5.yaml"
import yaml


class DotDict(dict):
    def __init__(self, mapping=None):
        super().__init__()
        mapping = mapping or {} 
        for key, value in mapping.items():
            self[key] = DotDict(value) if isinstance(value, dict) else value

    def __getattr__(self, key):
        try:
            return self[key]
        except KeyError:
            raise AttributeError(f"Key '{key}' not in config")

    def __setattr__(self, key, value):
        self[key] = value

def load_yaml_as_dotdict(filepath):
    with open(filepath, "r") as file:
        data = yaml.safe_load(file) or {}  # for if yaml empty lined
    return DotDict(data)

cm = load_yaml_as_dotdict(yaml_path)

In [3]:
model = FluidGPT_B(emb_dim=96,
                    data_dim=[64, cm.temporal_bundling, cm.in_channels, 128, 128],
                    patch_size=(cm.patch_size, cm.patch_size),
                    hiddenout_dim=cm.hiddenout_dim,
                    depth=cm.depth,
                    stage_depths=cm.stage_depths,
                    num_heads=cm.num_heads,
                    window_size=cm.window_size,
                    use_flex_attn=cm.use_flex_attn
                    ).cuda()
model.eval()
# Load the model weights
checkpoint_path = "C:/Users/20183172/Documents/2024-II/prep/epoch=28-step=563673.ckpt"
checkpoint = torch.load(checkpoint_path, map_location="cuda")
state_dict = checkpoint["state_dict"]
new_state_dict = {k.replace("model.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

c:\Users\20183172\AppData\Local\anaconda3\envs\grad311\Lib\site-packages\torch\functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:4316.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

In [5]:
x = torch.randn(16, 5, 2, 128, 128).cuda()
with torch.no_grad():
    y = model(x)

In [ ]:
from dataloaders.DiskDatasetDiv import DiskDatasetDiv
from torch.utils.data import DataLoader

#dataset = DiskDatasetDiv(preproc_path='../datasets/prjs1359/preproc_amira-s6', temporal_bundling=1)
dataset = DiskDatasetDiv(preproc_path='/datasets/preproc_pdegym-test', temporal_bundling=2)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False, num_workers=16, pin_memory=True, persistent_workers=True)
dataset.avgnorm = dataset.avg
dataset.stdnorm = dataset.std

dataset.avgnorm, dataset.stdnorm


In [ ]:
for i, (x, y) in enumerate(dataloader):
    x = x.cuda()
    y = y.cuda()
    with torch.no_grad():
        out = model(x)
    print(f"Batch {i+1}: Output shape: {out.shape}, Target shape: {y.shape}")
    if i == 0:
        break  # Just test the first batch
import matplotlib.pyplot as plt
import numpy as np

# plot the x and y 
fig, ax = plt.subplots(3, 3, figsize=(12, 6))
ax[0, 0].imshow(x[0, 0].cpu().numpy(), cmap='gray')
ax[0, 1].imshow(y[0, 0].cpu().numpy(), cmap='gray')
ax[1, 0].imshow(out[0, 0].cpu().numpy(), cmap='gray')

NameError: name 'dataloader' is not defined